SVM proves a failure for this dataset (R<sup>2</sup> < 0), perhaps for the enormous scale or many variables or complex associations among variables. 

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')
path = "/content/gdrive/My Drive/Colab Notebooks"

Mounted at /content/gdrive/


In [6]:
import re, os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [7]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR, SVR
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [9]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

from fastai.tabular import *
from fastai.tabular.all import *
from fastai.tabular.data import *
from fastai.tabular.learner import *

     |████████████████████████████████| 727kB 20.3MB/s 
     |████████████████████████████████| 1.2MB 54.3MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 194kB 50.3MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
data = pd.read_csv(path+"/data_shuf.csv")
alist = data.columns.to_list()
alist

['pop2012',
 'road_len',
 'cen_dist',
 'city_pop_total',
 'night2012',
 'land_cover']

In [12]:
data.describe()

,pop2012,road_len,cen_dist,city_pop_total,night2012,land_cover
count,433774.000000,433774.000000,433774.000000,433774.000000,433774.000000,433774.000000
mean,0.008004,0.071480,0.214999,0.192048,0.020648,6.906518
std,0.032205,0.093311,0.142153,0.301218,0.042011,1.857292
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.115120,0.021847,0.001736,7.000000
50%,0.000061,0.046930,0.182038,0.036372,0.004673,7.000000
75%,0.001265,0.104343,0.279313,0.141055,0.017724,8.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000


In [13]:
cat_names = alist[-1]
cont_names = alist[1:-1]
y_names = alist[0]
procs = [Categorify,]

In [14]:
percent, length = 0.8, len(data.axes[0])
stop = int(percent * length)
splits = [list(range(stop)), list(range(stop,length))]

In [15]:
cont,cat = cont_cat_split(data, 1, dep_var=y_names)
to = TabularPandas(data, procs, cat, cont, y_names=y_names, splits=splits)

In [16]:
len(to.xs),len(to.train),len(to.valid)

(433774, 347019, 86755)

In [17]:
xs, y = to.xs,to.y
train_xs,train_y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

## Linear SVM

In [22]:
def rmse(y,y_pred): return mean_squared_error(y,y_pred,squared=False)

In [23]:
svm_reg = LinearSVR(epsilon=0.5)

In [24]:
import time
start = time.time()
m = svm_reg.fit(train_xs, train_y);
end = time.time()
print("Time:", end-start)

Time: 1.075394868850708


In [25]:
train_rmse = rmse(m.predict(train_xs), train_y)
valid_rmse = rmse(m.predict(valid_xs), valid_y)

In [26]:
std = data.describe().to_dict()['pop2012']['std']
R2 = 1 - ( valid_rmse / std ) ** 2
train_rmse, valid_rmse, R2

(0.3693885851965111, 0.3694745086054532, -130.62172866592584)

## Gaussian Radial Basis Function (RBF)

In [27]:
# https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html
rbf_reg = SVR(kernel='rbf', gamma=0.1, C=1000)

In [28]:
import time
start = time.time()
m = rbf_reg.fit(train_xs, train_y);
end = time.time()
print("Time:", end-start)

Time: 1074.308501958847


In [39]:
train_rmse = rmse(m.predict(train_xs), train_y)
valid_rmse = rmse(m.predict(valid_xs), valid_y)

In [40]:
R2 = 1 - ( valid_rmse / std ) ** 2
train_rmse, valid_rmse, R2

(0.06930911787129079, 0.06928333240690263, -3.628243145091081)

After multiple trails, I have given up in this model.